In [1]:
import numpy as np
import pickle
from sklearn import metrics
import re
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import os
import pandas as pd

from ast import literal_eval

from process_predictions import get_predictions
from utils import assess_performance, flatten, compare_preds

In [2]:
all_cols = [
    'present_prim_tags',
    'present_sec_tags',
    'sectors',
    'pillars_1d',
    'pillars_2d',
    'subpillars_2d_part1', 
   'subpillars_2d_part2',
   'subpillars_1d_part1',
   'subpillars_1d_part2',
   'subpillars_1d_part3',
    'gender',
    'age',
    'specific_needs_groups',
    'affected_groups_levels_2_3',
    'gender_snorkel',
    'subpillars_1d',
    'subpillars_2d'
    ]

In [3]:
DATA_PATH = os.path.join(
    '..', '..', '..', '..', "data", "frameworks_data", 'data_v0.7.1'
)


test_df = pd.read_csv(os.path.join(DATA_PATH, 'new_columns_test_v0.7.1.csv'))


for col in all_cols:

    test_df[col] = test_df[col].apply(
        lambda x: [item for item in literal_eval(x) if item!='NOT_MAPPED']
    )   
    
    

/home/selim/anaconda3/envs/deepl/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
trained_columns = ['sectors', 'subpillars_2d', 'subpillars_1d']

In [5]:
pickle_file = open("18_nov_2models_run.pickle", "rb") ##NAME OF PICKLE FILE
doc = pickle.load(pickle_file)
doc['thresholds'].keys()

dict_keys(['sectors', 'present_prim_tags', 'pillars_2d', 'pillars_1d', 'impact_capresp_humcond', 'need_intervention_risk', 'context_covid', 'displacement_shockevent', 'access_infcom_casualities'])

In [6]:

final_preds = get_predictions(
    doc['preds'], 
    doc['thresholds'], 
    trained_columns,
    subpillars_2d_cols=['impact_capresp_humcond', 'need_intervention_risk'],
    subpillars_1d_cols=['context_covid', 'displacement_shockevent', 'access_infcom_casualities'],
    prim_tags=True,
    nb_entries=len(test_df), 
    ratio_nb=1.05)
     

In [7]:
predictions_df = test_df[['excerpt']]
predictions_df['sectors'] = final_preds['sectors']
predictions_df['subpillars_2d'] = final_preds['subpillars_2d']
predictions_df['subpillars_1d'] = final_preds['subpillars_1d']
predictions_df.to_csv('predictions_model_prim_tags_v1.csv', index=None)

<ipython-input-7-e1ae16b014aa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions_df['sectors'] = final_preds['sectors']
<ipython-input-7-e1ae16b014aa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions_df['subpillars_2d'] = final_preds['subpillars_2d']
<ipython-input-7-e1ae16b014aa>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [10]:
cols = trained_columns
raw_results = {}
tot_wrong_ids = {}
for col in cols:
    if col=='sectors':
        predictions_df = test_df[['excerpt']]
        predictions_df['sectors'] = final_preds['sectors']
        ids_not_cross = test_df.sectors.apply(lambda x: 'Cross' not in x)
        gt_sectors = test_df[ids_not_cross].sectors.tolist()
        preds_sectors = predictions_df[ids_not_cross].sectors.tolist()
        preds = compare_preds(gt_sectors, preds_sectors)
        results_col = preds['scores']
    else:    
        preds = compare_preds(test_df[col].values, final_preds[col])
        results_col = preds['scores']

    raw_results[col] = results_col
    
df_results_raw = pd.DataFrame.from_dict(raw_results, orient='columns')
df_results_raw
#df_results_raw.iloc[[0, 3,4]]

<ipython-input-10-fbbd09d45e1f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions_df['sectors'] = final_preds['sectors']


,sectors,subpillars_2d,subpillars_1d
proportion_perfect_matches,0.66,0.47,0.7
proportion_at_least_one_false,0.34,0.53,0.3
proportion_at_leaset_one_missing,0.17,0.36,0.2


In [11]:
results_subpillars_2d = assess_performance (
    final_preds['subpillars_2d'], 
    test_df['subpillars_2d'].tolist(), 
    sorted(list(set(flatten(test_df['subpillars_2d'])))),
    'subpillars_2d'
    )
results_subpillars_2d

,macro_precision,macro_recall,macro_f1_score,1_precision,0_precision,1_recall,0_recall,1_f1_score,0_f1_score,hamming_loss
At Risk->Number Of People At Risk,0.499000,0.500000,0.500000,0.000000,0.999000,0.000000,1.000000,0.000000,0.999000,0.001000
At Risk->Risk And Vulnerabilities,0.708000,0.701000,0.705000,0.455000,0.961000,0.439000,0.964000,0.447000,0.963000,0.070000
Capacities & Response->International Response,0.826000,0.823000,0.824000,0.687000,0.965000,0.679000,0.966000,0.683000,0.965000,0.063000
Capacities & Response->Local Response,0.499000,0.500000,0.499000,0.000000,0.998000,0.000000,1.000000,0.000000,0.999000,0.002000
Capacities & Response->National Response,0.701000,0.685000,0.693000,0.428000,0.974000,0.394000,0.977000,0.410000,0.975000,0.047000
Capacities & Response->Number Of People Reached/Response Gaps,0.810000,0.795000,0.802000,0.633000,0.987000,0.601000,0.989000,0.617000,0.988000,0.023000
Humanitarian Conditions->Coping Mechanisms,0.773000,0.724000,0.746000,0.566000,0.981000,0.460000,0.987000,0.507000,0.984000,0.031000
Humanitarian Conditions->Living Standards,0.736000,0.737000,0.736000,0.567000,0.905000,0.569000,0.904000,0.568000,0.905000,0.156000
Humanitarian Conditions->Number Of People In Need,0.777000,0.698000,0.731000,0.560000,0.994000,0.399000,0.997000,0.466000,0.996000,0.009000
Humanitarian Conditions->Physical And Mental Well Being,0.784000,0.751000,0.766000,0.623000,0.945000,0.541000,0.960000,0.579000,0.953000,0.085000


In [12]:
results_subpillars_1d = assess_performance (
    final_preds['subpillars_1d'], 
    test_df['subpillars_1d'].tolist(), 
    sorted(list(set(flatten(test_df['subpillars_1d'])))),
    'subpillars_1d'
    )
results_subpillars_1d

,macro_precision,macro_recall,macro_f1_score,1_precision,0_precision,1_recall,0_recall,1_f1_score,0_f1_score,hamming_loss
Casualties->Dead,0.822000,0.793000,0.807000,0.653000,0.991000,0.593000,0.993000,0.621000,0.992000,0.01600
Casualties->Injured,0.735000,0.749000,0.742000,0.471000,0.998000,0.500000,0.998000,0.485000,0.998000,0.00400
Casualties->Missing,0.749000,0.620000,0.662000,0.500000,0.999000,0.241000,1.000000,0.326000,0.999000,0.00200
Context->Demography,0.733000,0.646000,0.679000,0.480000,0.987000,0.299000,0.994000,0.368000,0.990000,0.01900
Context->Economy,0.809000,0.753000,0.778000,0.635000,0.982000,0.517000,0.989000,0.570000,0.986000,0.02800
Context->Environment,0.660000,0.622000,0.639000,0.326000,0.995000,0.248000,0.996000,0.282000,0.996000,0.00900
Context->Legal & Policy,0.709000,0.682000,0.695000,0.428000,0.990000,0.373000,0.992000,0.398000,0.991000,0.01800
Context->Politics,0.710000,0.738000,0.723000,0.427000,0.993000,0.484000,0.991000,0.454000,0.992000,0.01500
Context->Security & Stability,0.759000,0.729000,0.743000,0.543000,0.974000,0.478000,0.980000,0.508000,0.977000,0.04400
Context->Socio Cultural,0.633000,0.574000,0.594000,0.275000,0.991000,0.151000,0.996000,0.195000,0.993000,0.01300


In [13]:
predictions_df = test_df[['entry_id']]
predictions_df['sectors'] = final_preds['sectors']

ids_not_cross = test_df.sectors.apply(lambda x: 'Cross' not in x)
gt_sectors = test_df[ids_not_cross].sectors.tolist()
preds_sectors = predictions_df[ids_not_cross].sectors.tolist()
#results_col = compare_preds(gt_sectors, preds_sectors)

assess_performance (
    preds_sectors, 
    gt_sectors, 
    sorted(list(set(flatten(gt_sectors)))),
    'sectors'
    )

<ipython-input-13-ddc3440e36a9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions_df['sectors'] = final_preds['sectors']


,macro_precision,macro_recall,macro_f1_score,1_precision,0_precision,1_recall,0_recall,1_f1_score,0_f1_score,hamming_loss
Agriculture,0.7720,0.8360,0.8010,0.5510,0.9930,0.6840,0.9880,0.6100,0.9910,0.0180
Education,0.9070,0.9140,0.9100,0.8260,0.9880,0.8410,0.9870,0.8330,0.9880,0.0230
Food Security,0.8930,0.9040,0.8980,0.8050,0.9810,0.8320,0.9770,0.8180,0.9790,0.0380
Health,0.8330,0.8700,0.8490,0.7170,0.9500,0.8360,0.9040,0.7720,0.9260,0.1120
Livelihoods,0.8140,0.8440,0.8280,0.6600,0.9670,0.7330,0.9550,0.6950,0.9610,0.0690
Logistics,0.8030,0.7910,0.7970,0.6150,0.9910,0.5910,0.9920,0.6030,0.9920,0.0170
Nutrition,0.8770,0.9270,0.9000,0.7580,0.9950,0.8640,0.9900,0.8080,0.9930,0.0140
Protection,0.8240,0.8420,0.8330,0.7030,0.9450,0.7550,0.9290,0.7280,0.9370,0.1020
Shelter,0.8460,0.8700,0.8570,0.7050,0.9860,0.7580,0.9820,0.7300,0.9840,0.0300
WASH,0.9100,0.9210,0.9150,0.8320,0.9880,0.8570,0.9850,0.8440,0.9870,0.0250
